<a href="https://colab.research.google.com/github/hotdog1029/deeplearning/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC_%EB%B0%8F_Pre_trained_Embedding_Vector%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_Vocabulary_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torchtext import data
from torchtext import datasets

# Data Setting
TEXT = data.Field(batch_first = True,
                  fix_length = 500, # Sentence의 길이를 미리 제한하는 옵션
                  tokenize = str.split, # Tokenize를 설정하는 옵션 기본값은 띄어쓰기 기반의 파이썬의 string.split 함수
                  pad_first = True, # fix_length 대비 짧은 문장인 경의 Padding을 해야 하는데 Padding을 앞어세 중 것인지에 대한 옵션
                  pad_token = '[PAD]', # 위에서 설명한 Padding에 대한 특수 Token 설정
                  unk_token = '[UNK]') # Token Dictionary에 없는 Token이 나왔을 경우 해당 Token을 표현하는 특수 Token
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(text_field = TEXT,
                                             label_field = LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:09<00:00, 8.42MB/s]


In [7]:
# Data Length
print(f'Train Data Length: {len(train_data.examples)}')
print(f'Test Data Length: {len(test_data.examples)}')

# Data Fields
print(train_data.fields)

# Data Sample
print('---- Data Sample ----')
print('Input: ')
print(' '.join(vars(train_data.examples[1])['text']),'\\n')
print('Label: ')
print(vars(train_data.examples[1])['label'])

Train Data Length: 25000
Test Data Length: 25000
{'text': <torchtext.data.field.Field object at 0x7f7fc4618240>, 'label': <torchtext.data.field.LabelField object at 0x7f7fc4618198>}
---- Data Sample ----
Input: 
I have to point out, before you read this review, that in no way, is this a statement against Iranian people ... if you really want to read something into it, than hopefully you see, that I'm against politicians in general ... but if you're looking to be offended ... I can't help you!<br /><br />Not in Iran as this movie is banned there (see IMDb trivia for this movie). Which is a shame, because the movie is great. Would it not be for "Grbavica", this movie would have won at the International Film Festival in Berlin.<br /><br />Rightfully so (it was the runner-up, or second place if you will). Why? Because it is a movie about oppression. It's not even that this is a complete women issue. It is about the government trying to keep the people down. An analogy so clear that the gov

data.examples를 이용하면 데이터의 개수를 확인할 수 있고 vars() 함수를 이용하면 데이터 값을 직접 확인해볼 수 있다. Text Data는 Tokenize를 하기 전에 Data를 살펴보고 Cleansing 작업을 해야 한다. 다음과 같은 간단한 Data Cleansing 작업을 진행하였다.

In [11]:
import re

def PreProcessingText(input_sentence):
  input_sentence = input_sentence.lower() # 소문자화
  input_sentence = re.sub('<[^>]', repl = ' ', string = input_sentence) # '<br />' 처리
  input_sentence = re.sub('[!"#$%&\\()*+,-./:;<=>?@[\\\\]^_{|}~]', repl = ' ', string = input_sentence) # 특수 문자 처리
  input_sentence = re.sub('\\s+', repl = ' ', string = input_sentence) # 연속된 띄어쓰기 처리
  if input_sentence:
    return input_sentence

for example in train_data.examples:
  vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()
for example in test_data.examples:
  vars(example)['text'] = PreProcessingText(' '.join(vars(example)['text'])).split()

# pre-trained
TEXT.build_vocab(train_data,
                 min_freq = 2, # Vocab에 해당하는 Token에 최소한으로 등장하는 횟수를 제한을 둘 수 있다.
                 max_size = None, # 위와 같이 Token의 최소 등장 횟수로 Vocab의 Size 조절을 하는 것 외에 전체 Vocab Size 자체에도 제한을 둘 수 있다.
                 vectors = 'glove.6B.300d') # Pre-Trained Vector를 가져와 Vocab에 세팅하는 옵션, 원하는 Embedding을 정해 string 형태로 설정하면 된다.
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 399773/400000 [00:44<00:00, 8865.10it/s]

In [15]:
# Vocabulary Info
print(f'Vocab Size: {len(TEXT.vocab)}')

print('Vocab Examples: ')
for idx,(k,v) in enumerate(TEXT.vocab.stoi.items()):
  if idx >= 10:
    break
  print('\t', k ,v)
print('-------------------')
# Label Info
print(f'Lable Size: {len(LABEL.vocab)}')

print('Label Example: ')
for idx,(k,v) in enumerate(LABEL.vocab.stoi.items()):
  print('\t',k,v)

# Check embedding vectors
print(TEXT.vocab.vectors.shape)

Vocab Size: 97856
Vocab Examples: 
	 [UNK] 0
	 [PAD] 1
	 the 2
	 a 3
	 and 4
	 of 5
	 to 6
	 is 7
	 r 8
	 in 9
-------------------
Lable Size: 2
Label Example: 
	 neg 0
	 pos 1
torch.Size([97856, 300])


이제 Valieation 구분과 Iterator를 이용해 Batch Data를 만들면 모델 학습을 위한 데이터 설정은 모두 끝난다

In [16]:
import random

# Spliting Valid set
train_data, valid_data = train_data.split(random_state = random.seed(0),
                                          split_ratio = 0.8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(datasets=(train_data, valid_data, test_data),
                                                                           batch_size = 30,
                                                                           device = device) 